In this exercise, use NMF to recommender popular music artists! 

we have 2 scv files

1. 100 artist name

2. 24800 samples include 500 user

that merge those toghter and create sparse array artists rows whose rows correspond to artists and columns whose correspond to users. The entries give the number of times each artist was listened to by each user.**

In this exercise, build a pipeline and transform the array into normalized NMF features. The first step in the pipeline, MaxAbsScaler, transforms the data so that all users have the same influence on the model, regardless of how many different artists they've listened to.

then we use the resulting normalized NMF features for recommendation with NMF and cosine similarities!

Perform the necessary imports


In [1]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer, MaxAbsScaler
from sklearn.pipeline import make_pipeline

In [2]:
df = pd.read_csv('scrobbler-small-sample.csv')
df.head()

,user_offset,artist_offset,playcount
0,1,79,58
1,1,84,80
2,1,86,317
3,1,89,64
4,1,96,159


In [3]:
artists_df=pd.read_csv('artists.csv', header=None, names=['artist_name'])
artist_names=artists_df['artist_name']
artists_df.head()

,artist_name
0,Massive Attack
1,Sublime
2,Beastie Boys
3,Neil Young
4,Dead Kennedys


then add artist_offset column to artists_df to have same column that merge with df

In [4]:
artists_df['artist_offset'] = artists_df.index
artists_df.head()

,artist_name,artist_offset
0,Massive Attack,0
1,Sublime,1
2,Beastie Boys,2
3,Neil Young,3
4,Dead Kennedys,4


then, we merge the 2 data frames obtained from csv files together. To have a single data frame

In [5]:
temp_df=pd.merge(df,artists_df,on="artist_offset")

In [6]:
temp_df.head()

,user_offset,artist_offset,playcount,artist_name
0,1,79,58,Misfits
1,5,79,92,Misfits
2,68,79,86,Misfits
3,189,79,113,Misfits
4,206,79,244,Misfits


In [7]:
artists=temp_df.pivot_table(columns = 'user_offset',index = 'artist_name',values='playcount',fill_value=0)
artists.head()

user_offset,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
artist_name,,,,,,,,,,,,,,,,,,,,,
2Pac,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50 Cent,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC/DC,0,0,0,0,0,0,0,0,162,0,...,0,0,0,0,0,0,0,0,0,0
Aimee Mann,0,0,120,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alice in Chains,0,0,0,0,0,0,0,0,575,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Create a MaxAbsScaler: scaler
scaler = MaxAbsScaler()

# Create an NMF model: nmf
nmf = NMF(n_components=20)

# Create a Normalizer: normalizer
normalizer = Normalizer()

# Create a pipeline: pipeline
pipeline = make_pipeline(scaler ,nmf ,normalizer)

# Apply fit_transform to artists: norm_features
norm_features = pipeline.fit_transform(artists)


In [9]:
# Create a DataFrame: df
df = pd.DataFrame(norm_features,index=artist_names)

In [10]:
# Select row of 'Bruce Springsteen': artist
artist = df.loc['Bruce Springsteen']

# Compute cosine similarities: similarities
similarities = df.dot(artist)

# Display those with highest cosine similarity
print(similarities.nlargest())

artist_name
Bruce Springsteen    1.000000
Rufus Wainwright     0.922892
AC/DC                0.922790
Aimee Mann           0.920036
Foo Fighters         0.906408
dtype: float64
